In [10]:
import os
os.chdir(os.path.expanduser("~/Dev/personal/is_it_ai"))
from utils.data_ops import get_whole_dataset
import gc
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.metrics import roc_auc_score
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

from datasets import Dataset
from tqdm.auto import tqdm
from transformers import PreTrainedTokenizerFast

from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
import optuna
import gc

In [11]:
train = pd.read_csv("data/train_v2_drcat_02.csv", sep=',')
test = pd.read_csv('data/test_essays.csv')
train = train.drop_duplicates(subset=['text'])
train.reset_index(drop=True, inplace=True)
X = train["text"]
y = train['label']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)
LOWERCASE = False
VOCAB_SIZE = 30522

In [12]:
raw_tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))
raw_tokenizer.normalizer = normalizers.Sequence([normalizers.NFC()] + [normalizers.Lowercase()] if LOWERCASE else [])
raw_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.BpeTrainer(vocab_size=VOCAB_SIZE, special_tokens=special_tokens)
dataset = Dataset.from_pandas(test[['text']])
def train_corp_iter(): 
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["text"]
raw_tokenizer.train_from_iterator(train_corp_iter(), trainer=trainer)
tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=raw_tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)
tokenized_texts_test = []

for text in tqdm(x_test):
    tokenized_texts_test.append(tokenizer.tokenize(text))

tokenized_texts_train = [] 
for text in tqdm(x_train):
    tokenized_texts_train.append(tokenizer.tokenize(text))

100%|██████████| 35894/35894 [00:44<00:00, 807.55it/s]


In [13]:
def dummy(text):
    return text
vectorizer = TfidfVectorizer(ngram_range=(3, 5), lowercase=False, sublinear_tf=True, analyzer = 'word',
    tokenizer = dummy,
    preprocessor = dummy,
    token_pattern = None, strip_accents='unicode')

vectorizer.fit(tokenized_texts_test)

# Getting vocab
vocab = vectorizer.vocabulary_

print(vocab)

vectorizer = TfidfVectorizer(ngram_range=(3, 5), lowercase=False, sublinear_tf=True, vocabulary=vocab,
                            analyzer = 'word',
                            tokenizer = dummy,
                            preprocessor = dummy,
                            token_pattern = None, strip_accents='unicode'
                            )

tf_train = vectorizer.fit_transform(tokenized_texts_train)
tf_test = vectorizer.transform(tokenized_texts_test)

del vectorizer
gc.collect()

{'Ġ [UNK] [UNK]': 12789, '[UNK] [UNK] Ġ': 3255, '[UNK] Ġ [UNK]': 5253, '[UNK] [UNK] [UNK]': 2722, '[UNK] [UNK] e': 3140, '[UNK] e Ġ': 4829, 'e Ġ C': 11381, 'Ġ C [UNK]': 12589, 'C [UNK] a': 1811, '[UNK] a [UNK]': 3392, 'a [UNK] [UNK]': 5718, '[UNK] e [UNK]': 4553, 'e [UNK] [UNK]': 10315, 'e Ġ [UNK]': 11401, 'Ġ [UNK] e': 13071, 'e Ġ a': 11506, 'Ġ a [UNK]': 13247, '[UNK] [UNK] a': 2871, 'a [UNK] Ġ': 6014, 'e Ġ e': 11640, 'Ġ e [UNK]': 13887, '[UNK] [UNK] .': 2570, '[UNK] . Ġ': 2148, '. Ġ [UNK]': 571, '[UNK] [UNK] c': 3001, '[UNK] c [UNK]': 3932, 'c [UNK] e': 8057, 'e [UNK] Ġ': 10623, 'a [UNK] e': 5944, '[UNK] [UNK] d': 3071, '[UNK] d Ġ': 4384, 'd Ġ b': 9876, 'Ġ b e': 13589, 'b e Ġ': 7545, '[UNK] Ġ b': 5431, 'b e c': 7506, 'e c a': 11017, 'c a [UNK]': 8184, 'e [UNK] e': 10552, '[UNK] Ġ a': 5369, 'Ġ a Ġ': 13423, 'a Ġ [UNK]': 6768, '[UNK] Ġ c': 5474, 'Ġ c [UNK]': 13658, 'c [UNK] [UNK]': 7893, 'd Ġ [UNK]': 9741, 'Ġ b [UNK]': 13498, 'b [UNK] [UNK]': 7030, 'c [UNK] a': 7966, 'e [UNK] .': 10266, 

422

In [14]:
clf = MultinomialNB(alpha=0.1)
sgd_model = SGDClassifier(max_iter=8000, tol=1e-4, loss="modified_huber") 
p6={'n_iter': 2500,
    'verbose': -1,
    'objective': 'cross_entropy',
    'metric': 'auc',
    'learning_rate': 0.01, 
    'colsample_bytree': 0.78,
    'colsample_bynode': 0.8, 
    'lambda_l1': 4.562963348932286, 
    'lambda_l2': 2.97485, 
    'min_data_in_leaf': 115, 
    'max_depth': 23, 
    'max_bin': 898}

lgb=LGBMClassifier(**p6)
# cat=CatBoostClassifier(iterations=2000,
#                         verbose=0,
#                         l2_leaf_reg=6.6591278779517808,
#                         learning_rate=0.1,
#                         subsample = 0.4,
#                         allow_const_label=True,loss_function = 'CrossEntropy')
weights = [0.068,0.311,0.31]

ensemble = VotingClassifier(estimators=[('mnb',clf)
                                        ('sgd', sgd_model),
                                        ('lgb',lgb), 
                                        # ('cat', cat)
                                        ],
                            weights=weights, voting='soft', n_jobs=-1)
ensemble.fit(tf_train, y_train)
gc.collect()
final_preds = ensemble.predict_proba(tf_test)[:,1]
final_preds

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

array([0.08980771, 0.00277223, 0.00687357, ..., 0.99990757, 0.99784991,
       0.00213039])

In [15]:
print(f"{np.mean(y_test - final_preds)} -> Accuracy")

-0.0051254059191815715 -> Accuracy
